### import libraries

In [1]:
from keras.layers import Conv3D, MaxPool3D, Flatten, Dense
from keras.layers import Dropout, Input, BatchNormalization
from sklearn.metrics import confusion_matrix, accuracy_score
from plotly.offline import iplot, init_notebook_mode
from keras.losses import categorical_crossentropy
from keras.optimizers import Adadelta
import plotly.graph_objs as go
from matplotlib.pyplot import cm
from keras.models import Model
import numpy as np
import keras
import h5py

init_notebook_mode(connected=True)
%matplotlib inline



Using TensorFlow backend.
/home/zahangir/.local/lib/python2.7/site-packages/requests/__init__.py:83: RequestsDependencyWarning: Old version of cryptography ([1, 2, 3]) may cause slowdown.
  warnings.warn(warning, RequestsDependencyWarning)


### Read 3D MNIST dataset

In [6]:
with h5py.File('full_dataset_vectors.h5', 'r') as dataset:
    x_train = dataset["X_train"][:]
    x_test = dataset["X_test"][:]
    y_train = dataset["y_train"][:]
    y_test = dataset["y_test"][:]

    
print ("x_train shape: ", x_train.shape)
print ("y_train shape: ", y_train.shape)

print ("x_test shape:  ", x_test.shape)
print ("y_test shape:  ", y_test.shape)

('x_train shape: ', (10000, 4096))
('y_train shape: ', (10000,))
('x_test shape:  ', (2000, 4096))
('y_test shape:  ', (2000,))


### Data preprocessing

In [7]:

## Introduce the channel dimention in the input dataset 
xtrain = np.ndarray((x_train.shape[0], 4096, 3))
xtest = np.ndarray((x_test.shape[0], 4096, 3))

## iterate in train and test, add the rgb dimention 
def add_rgb_dimention(array):
    scaler_map = cm.ScalarMappable(cmap="Oranges")
    array = scaler_map.to_rgba(array)[:, : -1]
    return array
for i in range(x_train.shape[0]):
    xtrain[i] = add_rgb_dimention(x_train[i])
for i in range(x_test.shape[0]):
    xtest[i] = add_rgb_dimention(x_test[i])

## convert to 1 + 4D space (1st argument represents number of rows in the dataset)
xtrain = xtrain.reshape(x_train.shape[0], 16, 16, 16, 3)
xtest = xtest.reshape(x_test.shape[0], 16, 16, 16, 3)

## convert target variable into one-hot
y_train = keras.utils.to_categorical(y_train, 10)
y_test = keras.utils.to_categorical(y_test, 10)


print ("x_train shape: ", xtrain.shape)
print ("y_train shape: ", y_train.shape)

print ("x_test shape:  ", xtest.shape)
print ("y_test shape:  ", y_test.shape)

('x_train shape: ', (10000, 16, 16, 16, 3))
('y_train shape: ', (10000, 10))
('x_test shape:  ', (2000, 16, 16, 16, 3))
('y_test shape:  ', (2000, 10))


### Construct an 3D CNN model

In [8]:
## input layer
input_layer = Input((16, 16, 16, 3))

## convolutional layers
conv_layer1 = Conv3D(filters=8, kernel_size=(3, 3, 3), activation='relu')(input_layer)
conv_layer2 = Conv3D(filters=16, kernel_size=(3, 3, 3), activation='relu')(conv_layer1)

## add max pooling to obtain the most imformatic features
pooling_layer1 = MaxPool3D(pool_size=(2, 2, 2))(conv_layer2)

conv_layer3 = Conv3D(filters=32, kernel_size=(3, 3, 3), activation='relu')(pooling_layer1)
conv_layer4 = Conv3D(filters=64, kernel_size=(3, 3, 3), activation='relu')(conv_layer3)
pooling_layer2 = MaxPool3D(pool_size=(2, 2, 2))(conv_layer4)

## perform batch normalization on the convolution outputs before feeding it to MLP architecture
pooling_layer2 = BatchNormalization()(pooling_layer2)
flatten_layer = Flatten()(pooling_layer2)

## create an MLP architecture with dense layers : 4096 -> 512 -> 10
## add dropouts to avoid overfitting / perform regularization
dense_layer1 = Dense(units=2048, activation='relu')(flatten_layer)
dense_layer1 = Dropout(0.4)(dense_layer1)
dense_layer2 = Dense(units=512, activation='relu')(dense_layer1)
dense_layer2 = Dropout(0.4)(dense_layer2)
output_layer = Dense(units=10, activation='softmax')(dense_layer2)

## define the model with input layer and output layer
model = Model(inputs=input_layer, outputs=output_layer)

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 16, 16, 16, 3)     0         
_________________________________________________________________
conv3d_1 (Conv3D)            (None, 14, 14, 14, 8)     656       
_________________________________________________________________
conv3d_2 (Conv3D)            (None, 12, 12, 12, 16)    3472      
_________________________________________________________________
max_pooling3d_1 (MaxPooling3 (None, 6, 6, 6, 16)       0         
_________________________________________________________________
conv3d_3 (Conv3D)            (None, 4, 4, 4, 32)       13856     
_________________________________________________________________
conv3d_4 (Conv3D)            (None, 2, 2, 2, 64)       55360     
_________________________________________________________________
max_pooling3d_2 (MaxPooling3 (None, 1, 1, 1, 64)       0         
__________

### Compile the model and start training.

In [10]:
model.compile(loss=categorical_crossentropy, optimizer=Adadelta(lr=0.1), metrics=['acc'])
model.fit(x=xtrain, y=y_train, batch_size=128, epochs=50, validation_split=0.2)


Train on 8000 samples, validate on 2000 samples
Epoch 1/50
8000/8000 [==============================] - 172s 21ms/step - loss: 2.1894 - acc: 0.2233 - val_loss: 1.9893 - val_acc: 0.2990
Epoch 2/50
8000/8000 [==============================] - 169s 21ms/step - loss: 1.7918 - acc: 0.3880 - val_loss: 1.6951 - val_acc: 0.4085
Epoch 3/50
8000/8000 [==============================] - 168s 21ms/step - loss: 1.4726 - acc: 0.5034 - val_loss: 1.7925 - val_acc: 0.3770
Epoch 4/50
8000/8000 [==============================] - 168s 21ms/step - loss: 1.2944 - acc: 0.5533 - val_loss: 1.7000 - val_acc: 0.4490
Epoch 5/50
8000/8000 [==============================] - 168s 21ms/step - loss: 1.1900 - acc: 0.5895 - val_loss: 1.2128 - val_acc: 0.5780
Epoch 6/50
8000/8000 [==============================] - 168s 21ms/step - loss: 1.1001 - acc: 0.6236 - val_loss: 1.3454 - val_acc: 0.5415
Epoch 7/50
8000/8000 [==============================] - 168s 21ms/step - loss: 1.0313 - acc: 0.6456 - val_loss: 1.1496 - val_acc: 

### Test the model.

In [11]:
score = model.evaluate(xtest, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

('Test loss:', 2.4064021339416506)
('Test accuracy:', 0.623)


### See the labels from prediction

In [ ]:
pred = model.predict(xtest)
pred = np.argmax(pred, axis=1)
pred